In [23]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class FencingTournamentScheduler:
    def __init__(self, events_data, config=None):
        """
        Initialize the fencing tournament scheduler.
        
        Parameters:
        events_data: pandas DataFrame or list of dicts with 'name' and 'participants' columns
        config: dict with tournament configuration
        """
        if isinstance(events_data, pd.DataFrame):
            self.events = events_data.to_dict('records')
        else:
            self.events = events_data
            
        # Default configuration
        default_config = {
            'days': 4,
            'hours_per_day': 12,
            'knockout_percentage': 0.7,
            'total_strips': 40,
            'gap_between_rounds': 45,  # minutes
            'max_round_duration': 180,  # minutes
            'time_slot_duration': 30   # minutes
        }
        
        if config:
            default_config.update(config)
        self.config = default_config
        
        # Match durations by weapon (in minutes)
        self.match_durations = {
            'saber': {'group': 5, 'knockout': 10},
            'foil': {'group': 8, 'knockout': 16},
            'epee': {'group': 8, 'knockout': 16}
        }
        
        self.total_referees = int(self.config['total_strips'] * 1.2)
        
        # Calculate total time slots
        total_minutes = self.config['days'] * self.config['hours_per_day'] * 60
        self.total_time_slots = math.ceil(total_minutes / self.config['time_slot_duration'])
        
    def get_weapon_type(self, event_name):
        """Determine weapon type from event name."""
        if '佩剑' in event_name:
            return 'saber'
        elif '花剑' in event_name:
            return 'foil'
        elif '重剑' in event_name:
            return 'epee'
        return 'foil'  # default
    
    def is_team_event(self, event_name):
        """Check if event is a team event."""
        return '团体' in event_name
    
    def calculate_groups(self, participants):
        """Calculate number of groups needed (6-7 people per group)."""
        if participants <= 6:
            return 1
        return math.ceil(participants / 6.5)
    
    def calculate_group_matches(self, participants):
        """Calculate total matches in group stage."""
        groups = self.calculate_groups(participants)
        total_matches = 0
        remaining = participants
        
        for i in range(groups):
            group_size = min(remaining, math.ceil(remaining / (groups - i)))
            total_matches += (group_size * (group_size - 1)) // 2
            remaining -= group_size
            
        return total_matches
    
    def calculate_knockout_data(self, participants, percentage):
        """Calculate knockout stage participants and matches."""
        knockout_participants = int(participants * percentage)
        
        # Find largest power of 2 <= knockout_participants
        power_of_2 = 1
        while power_of_2 * 2 <= knockout_participants:
            power_of_2 *= 2
        
        preliminary_matches = 0
        if knockout_participants > power_of_2:
            preliminary_matches = knockout_participants - power_of_2
        
        knockout_matches = power_of_2 - 1
        
        return {
            'participants': knockout_participants,
            'preliminary_matches': preliminary_matches,
            'knockout_matches': knockout_matches,
            'total_matches': preliminary_matches + knockout_matches
        }
    
    def calculate_referee_distribution(self):
        """Calculate optimal referee distribution by weapon type."""
        weapon_matches = {'saber': 0, 'foil': 0, 'epee': 0}
        
        for event in self.events:
            weapon = self.get_weapon_type(event['name'])
            
            if self.is_team_event(event['name']):
                weapon_matches[weapon] += event['participants'] - 1
                continue
            
            group_matches = self.calculate_group_matches(event['participants'])
            knockout_data = self.calculate_knockout_data(
                event['participants'], self.config['knockout_percentage']
            )
            weapon_matches[weapon] += group_matches + knockout_data['total_matches']
        
        total_matches = sum(weapon_matches.values())
        if total_matches == 0:
            return {'saber': 10, 'foil': 10, 'epee': 10}
        
        # Distribute referees proportionally
        referees = {}
        for weapon in weapon_matches:
            referees[weapon] = max(1, math.ceil(
                (weapon_matches[weapon] / total_matches) * self.total_referees
            ))
        
        # Adjust to ensure total doesn't exceed limit
        total = sum(referees.values())
        while total > self.total_referees:
            largest = max(referees.keys(), key=lambda k: referees[k])
            referees[largest] -= 1
            total -= 1
            
        return referees
    
    def calculate_round_requirements(self, matches, weapon, round_type, available_referees, available_strips):
        """Calculate strips, duration, and referees needed for a round."""
        match_duration = self.match_durations[weapon][round_type]
        max_concurrent_matches = min(matches, available_referees, available_strips)
        
        if max_concurrent_matches == 0 or matches == 0:
            return {
                'strips': 0,
                'duration': 0,
                'rounds': 0,
                'valid': True,
                'referees': 0
            }
        
        rounds = math.ceil(matches / max_concurrent_matches)
        total_duration = rounds * match_duration
        
        return {
            'strips': max_concurrent_matches,
            'duration': total_duration,
            'rounds': rounds,
            'valid': total_duration <= self.config['max_round_duration'],
            'referees': max_concurrent_matches
        }
    
    def create_time_slot_grid(self):
        """Create time slot grid for parallel scheduling."""
        grid = []
        for slot in range(self.total_time_slots):
            start_time = slot * self.config['time_slot_duration']
            day = (start_time // (self.config['hours_per_day'] * 60)) + 1
            
            grid.append({
                'slot_index': slot,
                'start_time': start_time,
                'end_time': start_time + self.config['time_slot_duration'],
                'day': day,
                'used_strips': 0,
                'used_referees': {'saber': 0, 'foil': 0, 'epee': 0},
                'events': []
            })
        return grid
    
    def can_schedule_in_slot(self, slot, event, round_info, referee_distribution):
        """Check if event can fit in time slot."""
        weapon = event['weapon']
        required_strips = round_info['strips']
        required_referees = round_info['referees']
        round_duration_slots = math.ceil(round_info['duration'] / self.config['time_slot_duration'])
        
        # Check if enough slots remain in day
        day_end = (slot['day'] * self.config['hours_per_day'] * 60) / self.config['time_slot_duration']
        slots_to_end = day_end - slot['slot_index']
        
        if round_duration_slots > slots_to_end:
            return False
        
        # Check resource availability
        available_strips = self.config['total_strips'] - slot['used_strips']
        available_referees = referee_distribution[weapon] - slot['used_referees'][weapon]
        
        return required_strips <= available_strips and required_referees <= available_referees
    
    def schedule_round_in_slot(self, time_grid, start_slot_index, event, round_info, round_type):
        """Schedule event round in time slot."""
        round_duration_slots = math.ceil(round_info['duration'] / self.config['time_slot_duration'])
        weapon = event['weapon']
        
        # Mark slots as used
        for i in range(round_duration_slots):
            slot = time_grid[start_slot_index + i]
            slot['used_strips'] += round_info['strips']
            slot['used_referees'][weapon] += round_info['referees']
            
            if i == 0:  # Only add event info to first slot
                # Get matches from round_info, with fallback to total_matches for knockout rounds
                matches = round_info.get('matches', round_info.get('total_matches', 0))
                
                slot['events'].append({
                    'event_name': event['name'],
                    'round_type': round_type,
                    'weapon': weapon,
                    'participants': event['participants'],
                    'strips': round_info['strips'],
                    'referees': round_info['referees'],
                    'matches': matches,
                    'duration': round_info['duration'],
                    'duration_slots': round_duration_slots
                })
        
        return {
            'start_slot': start_slot_index,
            'end_slot': start_slot_index + round_duration_slots - 1,
            'start_time': time_grid[start_slot_index]['start_time'],
            'end_time': time_grid[start_slot_index + round_duration_slots - 1]['end_time']
        }
    
    def find_available_slot(self, time_grid, event, round_info, referee_distribution, start_slot=0):
        """Find available time slot for event round."""
        round_duration_slots = math.ceil(round_info['duration'] / self.config['time_slot_duration'])
        
        for i in range(start_slot, len(time_grid) - round_duration_slots + 1):
            slot = time_grid[i]
            
            # Check if this slot and subsequent slots can accommodate the round
            can_fit = True
            for j in range(round_duration_slots):
                check_slot = time_grid[i + j]
                if not self.can_schedule_in_slot(check_slot, event, round_info, referee_distribution):
                    can_fit = False
                    break
                
                # Ensure we don't cross day boundaries
                if check_slot['day'] != slot['day']:
                    can_fit = False
                    break
            
            if can_fit:
                return i
        
        return None
    
    def process_events(self, referee_distribution):
        """Process all events and calculate their requirements."""
        processed_events = []
        
        for event in self.events:
            weapon = self.get_weapon_type(event['name'])
            event_type = 'team' if self.is_team_event(event['name']) else 'individual'
            
            processed_event = {
                'name': event['name'],
                'participants': event['participants'],
                'weapon': weapon,
                'type': event_type
            }
            
            if event_type == 'individual':
                # Calculate group stage
                group_matches = self.calculate_group_matches(event['participants'])
                group_req = self.calculate_round_requirements(
                    group_matches, weapon, 'group', 
                    referee_distribution[weapon], self.config['total_strips']
                )
                group_req['matches'] = group_matches
                
                # Calculate knockout stage
                knockout_data = self.calculate_knockout_data(
                    event['participants'], self.config['knockout_percentage']
                )
                knockout_req = self.calculate_round_requirements(
                    knockout_data['total_matches'], weapon, 'knockout',
                    referee_distribution[weapon], self.config['total_strips']
                )
                knockout_req.update(knockout_data)
                knockout_req['matches'] = knockout_data['total_matches']
                
                processed_event.update({
                    'group_stage': group_req,
                    'knockout': knockout_req,
                    'total_duration': group_req['duration'] + self.config['gap_between_rounds'] + knockout_req['duration'],
                    'max_strips': max(group_req['strips'], knockout_req['strips']),
                    'max_referees': max(group_req['referees'], knockout_req['referees'])
                })
            else:
                # Team event
                matches = event['participants'] - 1
                team_req = self.calculate_round_requirements(
                    matches, weapon, 'knockout',
                    referee_distribution[weapon], self.config['total_strips']
                )
                team_req['matches'] = matches
                
                # Find dependency
                individual_name = event['name'].replace('团体', '个人')
                
                processed_event.update({
                    'knockout': team_req,
                    'total_duration': team_req['duration'],
                    'max_strips': team_req['strips'],
                    'max_referees': team_req['referees'],
                    'dependency': individual_name
                })
            
            processed_events.append(processed_event)
        
        return processed_events
    
    def schedule_events_parallel(self, time_grid, processed_events, referee_distribution):
        """Schedule events using parallel allocation."""
        scheduled_events = {}
        unscheduled_events = []
        
        # Sort events: individual first, then by complexity
        sorted_events = sorted(processed_events, 
                             key=lambda x: (x['type'] == 'team', -x['participants']))
        
        # Schedule individual events first
        for event in sorted_events:
            if event['type'] == 'individual':
                scheduled = self.schedule_individual_event(time_grid, event, referee_distribution)
                if scheduled:
                    scheduled_events[event['name']] = scheduled
                else:
                    unscheduled_events.append({
                        'name': event['name'],
                        'reason': 'No available time slot',
                        'required_strips': event['max_strips'],
                        'required_referees': event['max_referees']
                    })
        
        # Schedule team events after dependencies
        for event in sorted_events:
            if event['type'] == 'team':
                dependency = scheduled_events.get(event['dependency'])
                if dependency:
                    scheduled = self.schedule_team_event(time_grid, event, dependency, referee_distribution)
                    if scheduled:
                        scheduled_events[event['name']] = scheduled
                    else:
                        unscheduled_events.append({
                            'name': event['name'],
                            'reason': 'No slot after dependency',
                            'dependency': event['dependency']
                        })
                else:
                    unscheduled_events.append({
                        'name': event['name'],
                        'reason': 'Dependency not scheduled',
                        'dependency': event['dependency']
                    })
        
        return scheduled_events, unscheduled_events
    
    def schedule_individual_event(self, time_grid, event, referee_distribution):
        """Schedule an individual event (group + knockout)."""
        # Find slot for group stage
        group_slot = self.find_available_slot(time_grid, event, event['group_stage'], referee_distribution, 0)
        if group_slot is None:
            return None
        
        group_scheduled = self.schedule_round_in_slot(time_grid, group_slot, event, event['group_stage'], 'Group Stage')
        
        # Find slot for knockout (after gap)
        gap_slots = math.ceil(self.config['gap_between_rounds'] / self.config['time_slot_duration'])
        knockout_search_start = group_scheduled['end_slot'] + gap_slots + 1
        
        knockout_slot = self.find_available_slot(time_grid, event, event['knockout'], referee_distribution, knockout_search_start)
        if knockout_slot is None:
            # Rollback group scheduling
            self.rollback_round_scheduling(time_grid, group_scheduled, event, event['group_stage'])
            return None
        
        knockout_scheduled = self.schedule_round_in_slot(time_grid, knockout_slot, event, event['knockout'], 'Knockout')
        
        return {
            'type': 'individual',
            'group_stage': group_scheduled,
            'knockout': knockout_scheduled,
            'end_slot': knockout_scheduled['end_slot']
        }
    
    def schedule_team_event(self, time_grid, event, dependency, referee_distribution):
        """Schedule a team event after its dependency."""
        search_start = dependency['end_slot'] + 1
        
        slot = self.find_available_slot(time_grid, event, event['knockout'], referee_distribution, search_start)
        if slot is None:
            return None
        
        scheduled = self.schedule_round_in_slot(time_grid, slot, event, event['knockout'], 'Team Knockout')
        
        return {
            'type': 'team',
            'knockout': scheduled,
            'end_slot': scheduled['end_slot']
        }
    
    def rollback_round_scheduling(self, time_grid, scheduled_round, event, round_info):
        """Rollback a scheduled round."""
        round_duration_slots = math.ceil(round_info['duration'] / self.config['time_slot_duration'])
        weapon = event['weapon']
        
        for i in range(round_duration_slots):
            slot = time_grid[scheduled_round['start_slot'] + i]
            slot['used_strips'] -= round_info['strips']
            slot['used_referees'][weapon] -= round_info['referees']
            
            if i == 0:
                slot['events'] = [e for e in slot['events'] if e['event_name'] != event['name']]
    
    def debug_event_calculations(self, event_name, participants, knockout_percentage):
        """Debug helper to show match calculations for an event."""
        weapon = self.get_weapon_type(event_name)
        is_team = self.is_team_event(event_name)
        
        print(f"\n🔍 DEBUG: {event_name}")
        print(f"   Weapon: {weapon}, Participants: {participants}, Team: {is_team}")
        
        if is_team:
            matches = participants - 1
            print(f"   Team elimination matches: {matches}")
        else:
            # Group stage
            groups = self.calculate_groups(participants)
            group_matches = self.calculate_group_matches(participants)
            print(f"   Groups: {groups}, Group matches: {group_matches}")
            
            # Knockout stage
            knockout_data = self.calculate_knockout_data(participants, knockout_percentage)
            print(f"   Knockout participants: {knockout_data['participants']}")
            print(f"   Preliminary matches: {knockout_data['preliminary_matches']}")
            print(f"   Knockout matches: {knockout_data['knockout_matches']}")
            print(f"   Total knockout matches: {knockout_data['total_matches']}")
        
        return True
    
    def format_time(self, minutes):
        """Format minutes into HH:MM format."""
        hours = minutes // 60
        mins = minutes % 60
        return f"{hours:02d}:{mins:02d}"
    
    def generate_schedule(self):
        """Generate the complete tournament schedule."""
        # Calculate referee distribution
        referee_distribution = self.calculate_referee_distribution()
        
        # Process all events
        processed_events = self.process_events(referee_distribution)
        
        # Create time slot grid
        time_grid = self.create_time_slot_grid()
        
        # Schedule events
        scheduled_events, unscheduled_events = self.schedule_events_parallel(
            time_grid, processed_events, referee_distribution
        )
        
        # Format output
        return self.format_schedule_output(
            time_grid, scheduled_events, unscheduled_events, referee_distribution
        )
    
    def format_schedule_output(self, time_grid, scheduled_events, unscheduled_events, referee_distribution):
        """Format the schedule output."""
        # Group by days
        days = {}
        for slot in time_grid:
            day = slot['day']
            if day not in days:
                days[day] = []
            days[day].append(slot)
        
        daily_schedules = []
        for day_num, slots in days.items():
            active_slots = [s for s in slots if s['events']]
            if not active_slots:
                continue
            
            day_events = []
            event_map = {}
            
            # Consolidate events
            for slot in active_slots:
                for event in slot['events']:
                    key = f"{event['event_name']}-{event['round_type']}"
                    if key not in event_map:
                        event_map[key] = {
                            'name': event['event_name'],
                            'weapon': event['weapon'],
                            'participants': event['participants'],
                            'round_type': event['round_type'],
                            'start_time': self.format_time(slot['start_time']),
                            'end_time': self.format_time(slot['start_time'] + event['duration']),
                            'duration_minutes': event['duration'],
                            'strips': event['strips'],
                            'referees': event['referees'],
                            'matches': event['matches']
                        }
                        day_events.append(event_map[key])
            
            # Calculate day statistics
            max_strips_used = max([s['used_strips'] for s in slots], default=0)
            max_referees = {
                weapon: max([s['used_referees'][weapon] for s in slots], default=0)
                for weapon in ['saber', 'foil', 'epee']
            }
            
            total_duration = max([s['end_time'] for s in active_slots], default=0)
            
            daily_schedules.append({
                'day': day_num,
                'total_duration_minutes': total_duration,
                'max_strips_used': max_strips_used,
                'max_referees': max_referees,
                'events': sorted(day_events, key=lambda x: x['start_time'])
            })
        
        # Calculate overall statistics
        max_strips_needed = max([d['max_strips_used'] for d in daily_schedules], default=0)
        max_referees_needed = {
            weapon: max([d['max_referees'][weapon] for d in daily_schedules], default=0)
            for weapon in ['saber', 'foil', 'epee']
        }
        
        return {
            'config': self.config,
            'referee_distribution': referee_distribution,
            'total_events': len(self.events),
            'scheduled_events': len(scheduled_events),
            'unscheduled_events': unscheduled_events,
            'max_strips_needed': max_strips_needed,
            'max_referees_needed': max_referees_needed,
            'daily_schedules': daily_schedules
        }

def load_tournament_data(file_path):
    """Load tournament data from Excel file."""
    df = pd.read_excel(file_path)
    # Assuming columns are: [Index, Event Name, Strip Count, Participants]
    events = []
    for _, row in df.iterrows():
        if pd.notna(row.iloc[1]) and pd.notna(row.iloc[3]) and row.iloc[1] != '组别':
            events.append({
                'name': row.iloc[1],
                'participants': int(row.iloc[3])
            })
    return events

def print_schedule(schedule):
    """Print the tournament schedule in a readable format."""
    print("=" * 80)
    print("🤺 FENCING TOURNAMENT SCHEDULE")
    print("=" * 80)
    
    # Configuration
    config = schedule['config']
    print(f"\n📋 CONFIGURATION:")
    print(f"   Days: {config['days']}")
    print(f"   Hours per day: {config['hours_per_day']}")
    print(f"   Total strips available: {config['total_strips']}")
    print(f"   Knockout percentage: {config['knockout_percentage']:.0%}")
    
    # Summary statistics
    print(f"\n📊 SUMMARY:")
    print(f"   Total events: {schedule['total_events']}")
    print(f"   Scheduled events: {schedule['scheduled_events']}")
    print(f"   Unscheduled events: {len(schedule['unscheduled_events'])}")
    print(f"   Max strips needed: {schedule['max_strips_needed']}")
    
    # Referee distribution
    ref_dist = schedule['referee_distribution']
    ref_needed = schedule['max_referees_needed']
    print(f"\n👨‍⚖️ REFEREE ALLOCATION:")
    print(f"   Saber: {ref_needed['saber']}/{ref_dist['saber']} referees")
    print(f"   Foil: {ref_needed['foil']}/{ref_dist['foil']} referees") 
    print(f"   Epee: {ref_needed['epee']}/{ref_dist['epee']} referees")
    print(f"   Total: {sum(ref_needed.values())}/{sum(ref_dist.values())} referees")
    
    # Unscheduled events
    if schedule['unscheduled_events']:
        print(f"\n⚠️  UNSCHEDULED EVENTS:")
        for event in schedule['unscheduled_events']:
            print(f"   - {event['name']}: {event['reason']}")
    
    # Daily schedules
    for day in schedule['daily_schedules']:
        print(f"\n📅 DAY {day['day']}")
        print("-" * 60)
        duration_hours = day['total_duration_minutes'] // 60
        duration_mins = day['total_duration_minutes'] % 60
        print(f"Duration: {duration_hours}h {duration_mins}m | Strips: {day['max_strips_used']} | Events: {len(day['events'])}")
        
        # Resource usage
        max_refs = day['max_referees']
        print(f"Referees used - Saber: {max_refs['saber']}, Foil: {max_refs['foil']}, Epee: {max_refs['epee']}")
        
        # Group events by time for parallel display
        time_groups = {}
        for event in day['events']:
            time_key = f"{event['start_time']}-{event['end_time']}"
            if time_key not in time_groups:
                time_groups[time_key] = []
            time_groups[time_key].append(event)
        
        for time_range, events in sorted(time_groups.items()):
            start_time, end_time = time_range.split('-')
            
            if len(events) == 1:
                event = events[0]
                weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                print(f"\n   {weapon_emoji.get(event['weapon'], '🤺')} {event['start_time']}-{event['end_time']} | {event['name']}")
                print(f"      {event['weapon'].upper()} • {event['participants']} participants • {event['round_type']}")
                print(f"      Strips: {event['strips']} | Referees: {event['referees']} | Matches: {event['matches']}")
            else:
                print(f"\n   🔄 {start_time}-{end_time} | {len(events)} PARALLEL EVENTS:")
                for event in events:
                    weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                    print(f"      {weapon_emoji.get(event['weapon'], '🤺')} {event['name']}")
                    print(f"         {event['weapon'].upper()} • {event['participants']} participants • {event['round_type']}")
                    print(f"         Strips: {event['strips']} | Refs: {event['referees']} | Matches: {event['matches']}")

# Example usage function
def run_tournament_scheduler(file_path=None, sample_data=None, config=None):
    """
    Run the tournament scheduler with either a file or sample data.
    
    Parameters:
    file_path: path to Excel file (optional)
    sample_data: list of events (optional, used if file_path not provided)
    config: tournament configuration dict (optional)
    """
    
    # Load data
    if file_path:
        events = load_tournament_data(file_path)
        print(f"Loaded {len(events)} events from {file_path}")
    elif sample_data:
        events = sample_data
        print(f"Using {len(events)} sample events")
    else:
        # Default sample data
        events = [
            {'name': 'U14女子重剑个人', 'participants': 63},
            {'name': 'U14女子佩剑个人', 'participants': 61},
            {'name': 'U16男子花剑个人', 'participants': 39},
            {'name': 'U12男子花剑个人', 'participants': 224},
            {'name': 'U10男子佩剑个人', 'participants': 80},
            {'name': 'U14女子重剑团体', 'participants': 5},
            {'name': 'U14女子佩剑团体', 'participants': 6}
        ]
        print(f"Using {len(events)} default sample events")
    
    # Default configuration
    default_config = {
        'days': 2,
        'hours_per_day': 8,
        'knockout_percentage': 1.0,
        'total_strips': 18,
        'gap_between_rounds': 45,
        'max_round_duration': 180,
        'time_slot_duration': 10
    }
    
    if config:
        default_config.update(config)
    
    # Create scheduler and generate schedule
    scheduler = FencingTournamentScheduler(events, default_config)
    schedule = scheduler.generate_schedule()
    
    # Print results
    print_schedule(schedule)
    
    return schedule

# Test function to verify calculations
def test_calculations():
    """Test the match calculations for debugging."""
    
    # Create a simple scheduler to test calculations
    test_events = [
        {'name': 'U12女子重剑个人', 'participants': 118},
        {'name': 'U10男子花剑个人', 'participants': 139}
    ]
    
    config = {'knockout_percentage': 0.7}
    scheduler = FencingTournamentScheduler(test_events, config)
    
    for event in test_events:
        scheduler.debug_event_calculations(
            event['name'], 
            event['participants'], 
            config['knockout_percentage']
        )
    
    return scheduler

# Jupyter notebook friendly function
def quick_schedule(events_data=None, days=2, hours_per_day=8, total_strips=18, debug=False):
    """Quick schedule generation for Jupyter notebook."""
    config = {
        'days': days,
        'hours_per_day': hours_per_day,
        'total_strips': total_strips,
        'knockout_percentage': 1.0
    }
    
    if debug:
        print("🔍 DEBUGGING MODE - Testing calculations...")
        test_scheduler = test_calculations()
        print("\n" + "="*60)
    
    return run_tournament_scheduler(sample_data=events_data, config=config)

In [24]:
tournament_events = [
    {'name': 'U10男子花剑个人', 'participants': 51},
    {'name': 'U10男子重剑个人', 'participants': 32},
    {'name': 'U14男子佩剑个人', 'participants': 7},
    {'name': 'U10女子花剑个人', 'participants': 44},
    {'name': 'U10女子重剑个人', 'participants': 32},
    {'name': 'U14女子佩剑个人', 'participants': 9},
    {'name': 'U14男子花剑个人', 'participants': 18},
    {'name': 'U14女子花剑个人', 'participants': 14},
    {'name': 'U14男子重剑个人', 'participants': 20},
    {'name': 'U10男子佩剑个人', 'participants': 43},
    {'name': 'U14女子重剑个人', 'participants': 15},
    {'name': 'U10女子佩剑个人', 'participants': 20},
    {'name': 'U12男子花剑个人', 'participants': 37},
    {'name': 'U12男子重剑个人', 'participants': 37},
    {'name': 'U12男子佩剑个人', 'participants': 26},
    {'name': 'U16女子花剑个人', 'participants': 5},
    {'name': 'U12女子花剑个人', 'participants': 32},
    {'name': 'U12女子佩剑个人', 'participants': 16},
    {'name': 'U12女子重剑个人', 'participants': 25},
    {'name': 'U16女子重剑个人', 'participants': 5},
    {'name': 'U16男子重剑个人', 'participants': 8},
    {'name': 'U16男子花剑个人', 'participants': 9},
]

# Custom configuration for maximum efficiency
config = {
    'days': 2,
    'hours_per_day': 8,
    'total_strips': 15,
    'knockout_percentage': 1.0,
    'gap_between_rounds': 30,  # Shorter gap
    'max_round_duration': 150  # Stricter time limit
}
schedule = quick_schedule(tournament_events, debug=True)

🔍 DEBUGGING MODE - Testing calculations...

🔍 DEBUG: U12女子重剑个人
   Weapon: epee, Participants: 118, Team: False
   Groups: 19, Group matches: 309
   Knockout participants: 82
   Preliminary matches: 18
   Knockout matches: 63
   Total knockout matches: 81

🔍 DEBUG: U10男子花剑个人
   Weapon: foil, Participants: 139, Team: False
   Groups: 22, Group matches: 372
   Knockout participants: 97
   Preliminary matches: 33
   Knockout matches: 63
   Total knockout matches: 96

Using 22 sample events
🤺 FENCING TOURNAMENT SCHEDULE

📋 CONFIGURATION:
   Days: 2
   Hours per day: 8
   Total strips available: 18
   Knockout percentage: 100%

📊 SUMMARY:
   Total events: 22
   Scheduled events: 17
   Unscheduled events: 5
   Max strips needed: 16

👨‍⚖️ REFEREE ALLOCATION:
   Saber: 5/5 referees
   Foil: 8/8 referees
   Epee: 8/8 referees
   Total: 21/21 referees

⚠️  UNSCHEDULED EVENTS:
   - U12男子花剑个人: No available time slot
   - U10女子重剑个人: No available time slot
   - U12女子花剑个人: No available time slot
   - 

In [27]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

@dataclass
class TimeSlot:
    """Represents a time slot with strip assignments"""
    start_time: int
    end_time: int
    day: int
    strip_assignments: Dict[int, Optional[str]]  # strip_id -> event_name
    referee_usage: Dict[str, int]  # weapon -> count
    
    def available_strips(self) -> List[int]:
        """Get list of available strip indices"""
        return [idx for idx, event in self.strip_assignments.items() if event is None]
    
    def assign_strips(self, event_name: str, strip_indices: List[int]) -> bool:
        """Assign specific strips to an event"""
        # Check if all requested strips are available
        for idx in strip_indices:
            if self.strip_assignments.get(idx) is not None:
                return False
        
        # Assign strips
        for idx in strip_indices:
            self.strip_assignments[idx] = event_name
        return True
    
    def release_strips(self, strip_indices: List[int]):
        """Release strips"""
        for idx in strip_indices:
            self.strip_assignments[idx] = None

@dataclass
class EventRound:
    """Represents a round of an event"""
    event_name: str
    round_type: str  # 'group', 'knockout', 'team'
    weapon: str
    participants: int
    matches: int
    duration_minutes: int
    required_strips: int
    required_referees: int
    priority: int  # Lower is higher priority

class OptimizedFencingScheduler:
    def __init__(self, events_data, config=None):
        """Initialize the optimized fencing tournament scheduler."""
        if isinstance(events_data, pd.DataFrame):
            self.events = events_data.to_dict('records')
        else:
            self.events = events_data
            
        # Default configuration
        default_config = {
            'days': 4,
            'hours_per_day': 12,
            'knockout_percentage': 0.7,
            'total_strips': 40,
            'min_gap_between_rounds': 30,  # minutes
            'max_gap_between_rounds': 60,  # minutes
            'max_round_duration': 180,  # minutes (3 hours)
            'time_slot_duration': 5,   # minutes (smaller for better granularity)
            'referee_multiplier': 1.2
        }
        
        if config:
            default_config.update(config)
        self.config = default_config
        
        # Match durations by weapon (in minutes)
        self.match_durations = {
            'saber': {'group': 5, 'knockout': 10},
            'foil': {'group': 8, 'knockout': 16},
            'epee': {'group': 8, 'knockout': 16}
        }
        
        self.total_referees = int(self.config['total_strips'] * self.config['referee_multiplier'])
        
        # Calculate total time slots
        total_minutes = self.config['days'] * self.config['hours_per_day'] * 60
        self.total_time_slots = total_minutes // self.config['time_slot_duration']
        
        # Initialize time grid
        self.time_grid = self._create_time_grid()
        
        # Track scheduled events
        self.scheduled_events = {}
        self.event_rounds = []
        
    def _create_time_grid(self) -> List[TimeSlot]:
        """Create optimized time slot grid with strip tracking"""
        grid = []
        for slot_idx in range(self.total_time_slots):
            start_time = slot_idx * self.config['time_slot_duration']
            end_time = start_time + self.config['time_slot_duration']
            day = (start_time // (self.config['hours_per_day'] * 60)) + 1
            
            # Initialize all strips as available
            strip_assignments = {i: None for i in range(self.config['total_strips'])}
            
            grid.append(TimeSlot(
                start_time=start_time,
                end_time=end_time,
                day=day,
                strip_assignments=strip_assignments,
                referee_usage={'saber': 0, 'foil': 0, 'epee': 0}
            ))
        return grid
    
    def get_weapon_type(self, event_name: str) -> str:
        """Determine weapon type from event name."""
        if '佩剑' in event_name:
            return 'saber'
        elif '花剑' in event_name:
            return 'foil'
        elif '重剑' in event_name:
            return 'epee'
        return 'foil'  # default
    
    def is_team_event(self, event_name: str) -> bool:
        """Check if event is a team event."""
        return '团体' in event_name
    
    def calculate_optimal_groups(self, participants: int) -> Tuple[int, List[int]]:
        """Calculate optimal group configuration to minimize variance"""
        if participants <= 7:
            return 1, [participants]
        
        # Try to make groups of 6 or 7 with minimal variance
        best_config = None
        min_variance = float('inf')
        
        for target_size in [6, 7]:
            num_groups = participants // target_size
            remainder = participants % target_size
            
            if remainder == 0:
                return num_groups, [target_size] * num_groups
            
            # Distribute remainder evenly
            group_sizes = [target_size] * num_groups
            for i in range(remainder):
                group_sizes[i % num_groups] += 1
            
            # Calculate variance
            variance = np.var(group_sizes)
            if variance < min_variance:
                min_variance = variance
                best_config = (len(group_sizes), group_sizes)
        
        return best_config
    
    def calculate_group_matches(self, participants: int) -> int:
        """Calculate total matches in group stage."""
        _, group_sizes = self.calculate_optimal_groups(participants)
        total_matches = sum(size * (size - 1) // 2 for size in group_sizes)
        return total_matches
    
    def calculate_knockout_structure(self, participants: int, percentage: float) -> Dict:
        """Calculate optimized knockout structure"""
        knockout_participants = int(participants * percentage)
        
        # Find the appropriate power of 2
        power_of_2 = 1
        while power_of_2 * 2 <= knockout_participants:
            power_of_2 *= 2
        
        # Calculate preliminary matches needed
        if knockout_participants > power_of_2:
            # Only bottom ranked fencers need preliminary
            preliminary_participants = 2 * (knockout_participants - power_of_2)
            preliminary_matches = knockout_participants - power_of_2
        else:
            preliminary_participants = 0
            preliminary_matches = 0
        
        # Main knockout rounds
        knockout_rounds = []
        remaining = power_of_2
        round_num = 1
        
        while remaining > 1:
            matches = remaining // 2
            knockout_rounds.append({
                'round': round_num,
                'matches': matches,
                'participants': remaining
            })
            remaining //= 2
            round_num += 1
        
        return {
            'total_participants': knockout_participants,
            'preliminary_matches': preliminary_matches,
            'preliminary_participants': preliminary_participants,
            'knockout_rounds': knockout_rounds,
            'total_knockout_matches': sum(r['matches'] for r in knockout_rounds),
            'total_matches': preliminary_matches + sum(r['matches'] for r in knockout_rounds)
        }
    
    def optimize_referee_distribution(self) -> Dict[str, int]:
        """Optimize referee distribution based on actual match load"""
        weapon_time_requirements = {'saber': 0, 'foil': 0, 'epee': 0}
        
        for event in self.events:
            weapon = self.get_weapon_type(event['name'])
            
            if self.is_team_event(event['name']):
                # Team events - single elimination
                matches = event['participants'] - 1
                duration = self.match_durations[weapon]['knockout']
                weapon_time_requirements[weapon] += matches * duration
            else:
                # Individual events
                # Group stage
                group_matches = self.calculate_group_matches(event['participants'])
                group_duration = self.match_durations[weapon]['group']
                weapon_time_requirements[weapon] += group_matches * group_duration
                
                # Knockout stage
                knockout_data = self.calculate_knockout_structure(
                    event['participants'], self.config['knockout_percentage']
                )
                knockout_duration = self.match_durations[weapon]['knockout']
                weapon_time_requirements[weapon] += knockout_data['total_matches'] * knockout_duration
        
        # Distribute referees proportionally with minimum guarantee
        total_time = sum(weapon_time_requirements.values())
        if total_time == 0:
            return {'saber': 10, 'foil': 10, 'epee': 10}
        
        referee_distribution = {}
        allocated = 0
        
        for weapon in ['saber', 'foil', 'epee']:
            if weapon_time_requirements[weapon] > 0:
                # Proportional allocation with minimum of 3 referees
                ideal_refs = (weapon_time_requirements[weapon] / total_time) * self.total_referees
                referee_distribution[weapon] = max(3, round(ideal_refs))
                allocated += referee_distribution[weapon]
            else:
                referee_distribution[weapon] = 0
        
        # Adjust if over-allocated
        while allocated > self.total_referees:
            # Reduce from weapon with most referees
            max_weapon = max(referee_distribution.keys(), key=lambda k: referee_distribution[k])
            if referee_distribution[max_weapon] > 3:
                referee_distribution[max_weapon] -= 1
                allocated -= 1
        
        # Distribute any remaining referees
        while allocated < self.total_referees:
            # Add to weapon with highest time requirement per referee
            best_weapon = max(
                [w for w in weapon_time_requirements if referee_distribution[w] > 0],
                key=lambda w: weapon_time_requirements[w] / referee_distribution[w]
            )
            referee_distribution[best_weapon] += 1
            allocated += 1
        
        return referee_distribution
    
    def calculate_round_requirements(self, matches: int, weapon: str, round_type: str,
                                   available_referees: int) -> Dict:
        """Calculate optimized round requirements"""
        if matches == 0:
            return {
                'strips_needed': 0,
                'duration': 0,
                'concurrent_matches': 0,
                'rounds': 0,
                'valid': True
            }
        
        match_duration = self.match_durations[weapon]['group' if round_type == 'group' else 'knockout']
        
        # Maximum concurrent matches limited by referees
        max_concurrent = min(matches, available_referees, self.config['total_strips'])
        
        # Optimize for minimum strips while respecting 3-hour constraint
        best_config = None
        min_strips = float('inf')
        
        for strips in range(1, max_concurrent + 1):
            rounds_needed = math.ceil(matches / strips)
            total_duration = rounds_needed * match_duration
            
            if total_duration <= self.config['max_round_duration']:
                if strips < min_strips:
                    min_strips = strips
                    best_config = {
                        'strips_needed': strips,
                        'duration': total_duration,
                        'concurrent_matches': strips,
                        'rounds': rounds_needed,
                        'valid': True
                    }
        
        if best_config is None:
            # If no valid config found, use maximum strips
            best_config = {
                'strips_needed': max_concurrent,
                'duration': math.ceil(matches / max_concurrent) * match_duration,
                'concurrent_matches': max_concurrent,
                'rounds': math.ceil(matches / max_concurrent),
                'valid': False
            }
        
        return best_config
    
    def find_optimal_time_slot(self, duration_slots: int, required_strips: int, 
                             weapon: str, required_referees: int, 
                             start_search: int = 0, same_day: bool = True,
                             preferred_day: Optional[int] = None) -> Optional[Tuple[int, List[int]]]:
        """Find optimal time slot and strip assignment"""
        current_day = self.time_grid[start_search].day if start_search < len(self.time_grid) else None
        
        for start_slot in range(start_search, len(self.time_grid) - duration_slots + 1):
            slot = self.time_grid[start_slot]
            
            # Check day constraint
            if same_day and current_day and slot.day != current_day:
                continue
            if preferred_day and slot.day != preferred_day:
                continue
            
            # Check if all slots in duration are in same day
            end_slot_idx = start_slot + duration_slots - 1
            if end_slot_idx >= len(self.time_grid):
                continue
            
            end_day = self.time_grid[end_slot_idx].day
            if slot.day != end_day:
                continue
            
            # Find contiguous strips available for entire duration
            available_strips_sets = []
            valid_allocation = True
            
            for slot_idx in range(start_slot, start_slot + duration_slots):
                current_slot = self.time_grid[slot_idx]
                
                # Check referee availability
                if current_slot.referee_usage.get(weapon, 0) + required_referees > \
                   self.referee_distribution.get(weapon, 0):
                    valid_allocation = False
                    break
                
                available_strips = current_slot.available_strips()
                if len(available_strips) < required_strips:
                    valid_allocation = False
                    break
                
                available_strips_sets.append(set(available_strips))
            
            if not valid_allocation:
                continue
            
            # Find strips available for entire duration
            common_strips = set.intersection(*available_strips_sets)
            if len(common_strips) >= required_strips:
                # Select lowest numbered strips for consistency
                selected_strips = sorted(list(common_strips))[:required_strips]
                return start_slot, selected_strips
        
        return None
    
    def allocate_time_slot(self, start_slot: int, duration_slots: int, 
                          event_name: str, weapon: str, strip_indices: List[int],
                          required_referees: int):
        """Allocate time slots and strips for an event"""
        for slot_idx in range(start_slot, start_slot + duration_slots):
            slot = self.time_grid[slot_idx]
            slot.assign_strips(event_name, strip_indices)
            slot.referee_usage[weapon] = slot.referee_usage.get(weapon, 0) + required_referees
    
    def deallocate_time_slot(self, start_slot: int, duration_slots: int,
                            weapon: str, strip_indices: List[int], required_referees: int):
        """Deallocate time slots and strips"""
        for slot_idx in range(start_slot, start_slot + duration_slots):
            slot = self.time_grid[slot_idx]
            slot.release_strips(strip_indices)
            slot.referee_usage[weapon] = max(0, slot.referee_usage.get(weapon, 0) - required_referees)
    
    def create_event_rounds(self) -> List[EventRound]:
        """Create all event rounds with priorities"""
        rounds = []
        
        # Process events by priority: individual events first, then team events
        individual_events = [e for e in self.events if not self.is_team_event(e['name'])]
        team_events = [e for e in self.events if self.is_team_event(e['name'])]
        
        # Sort individual events by participant count (larger events first for better packing)
        individual_events.sort(key=lambda x: -x['participants'])
        
        # Create rounds for individual events
        for idx, event in enumerate(individual_events):
            weapon = self.get_weapon_type(event['name'])
            
            # Group stage
            group_matches = self.calculate_group_matches(event['participants'])
            if group_matches > 0:
                group_req = self.calculate_round_requirements(
                    group_matches, weapon, 'group',
                    self.referee_distribution.get(weapon, 0)
                )
                
                rounds.append(EventRound(
                    event_name=event['name'],
                    round_type='group',
                    weapon=weapon,
                    participants=event['participants'],
                    matches=group_matches,
                    duration_minutes=group_req['duration'],
                    required_strips=group_req['strips_needed'],
                    required_referees=group_req['concurrent_matches'],
                    priority=idx * 2  # Even priorities for group stages
                ))
            
            # Knockout stage
            knockout_data = self.calculate_knockout_structure(
                event['participants'], self.config['knockout_percentage']
            )
            
            knockout_req = self.calculate_round_requirements(
                knockout_data['total_matches'], weapon, 'knockout',
                self.referee_distribution.get(weapon, 0)
            )
            
            rounds.append(EventRound(
                event_name=event['name'],
                round_type='knockout',
                weapon=weapon,
                participants=knockout_data['total_participants'],
                matches=knockout_data['total_matches'],
                duration_minutes=knockout_req['duration'],
                required_strips=knockout_req['strips_needed'],
                required_referees=knockout_req['concurrent_matches'],
                priority=idx * 2 + 1  # Odd priorities for knockout stages
            ))
        
        # Create rounds for team events
        for idx, event in enumerate(team_events):
            weapon = self.get_weapon_type(event['name'])
            matches = event['participants'] - 1
            
            team_req = self.calculate_round_requirements(
                matches, weapon, 'knockout',
                self.referee_distribution.get(weapon, 0)
            )
            
            rounds.append(EventRound(
                event_name=event['name'],
                round_type='team',
                weapon=weapon,
                participants=event['participants'],
                matches=matches,
                duration_minutes=team_req['duration'],
                required_strips=team_req['strips_needed'],
                required_referees=team_req['concurrent_matches'],
                priority=1000 + idx  # High priority number for team events
            ))
        
        return rounds
    
    def schedule_events(self) -> Tuple[Dict, List[Dict]]:
        """Schedule all events using constraint programming approach"""
        scheduled = {}
        unscheduled = []
        
        # Group rounds by event
        event_rounds_map = defaultdict(list)
        for round_obj in self.event_rounds:
            event_rounds_map[round_obj.event_name].append(round_obj)
        
        # Schedule individual events
        for event_name, rounds in event_rounds_map.items():
            if any(r.round_type == 'team' for r in rounds):
                continue  # Skip team events for now
            
            event_scheduled = self.schedule_individual_event(event_name, rounds)
            if event_scheduled:
                scheduled[event_name] = event_scheduled
            else:
                unscheduled.append({
                    'event': event_name,
                    'reason': 'Could not find suitable time slots',
                    'rounds': [r.round_type for r in rounds]
                })
        
        # Schedule team events after their dependencies
        for event_name, rounds in event_rounds_map.items():
            if not any(r.round_type == 'team' for r in rounds):
                continue
            
            # Find dependency (individual event)
            individual_event = event_name.replace('团体', '个人')
            if individual_event in scheduled:
                dependency = scheduled[individual_event]
                team_scheduled = self.schedule_team_event(
                    event_name, rounds[0], dependency
                )
                if team_scheduled:
                    scheduled[event_name] = team_scheduled
                else:
                    unscheduled.append({
                        'event': event_name,
                        'reason': 'Could not schedule after dependency',
                        'dependency': individual_event
                    })
            else:
                unscheduled.append({
                    'event': event_name,
                    'reason': 'Dependency not scheduled',
                    'dependency': individual_event
                })
        
        return scheduled, unscheduled
    
    def schedule_individual_event(self, event_name: str, rounds: List[EventRound]) -> Optional[Dict]:
        """Schedule an individual event (group + knockout)"""
        group_round = next((r for r in rounds if r.round_type == 'group'), None)
        knockout_round = next((r for r in rounds if r.round_type == 'knockout'), None)
        
        if not knockout_round:
            return None
        
        # Calculate slot requirements
        group_slots = 0
        group_result = None
        
        if group_round:
            group_slots = math.ceil(group_round.duration_minutes / self.config['time_slot_duration'])
            
            # Find slot for group stage
            slot_info = self.find_optimal_time_slot(
                group_slots, group_round.required_strips,
                group_round.weapon, group_round.required_referees, 0
            )
            
            if not slot_info:
                return None
            
            start_slot, strip_indices = slot_info
            
            # Allocate group stage
            self.allocate_time_slot(
                start_slot, group_slots, f"{event_name}-group",
                group_round.weapon, strip_indices, group_round.required_referees
            )
            
            group_result = {
                'start_slot': start_slot,
                'end_slot': start_slot + group_slots - 1,
                'strips': strip_indices,
                'duration': group_round.duration_minutes,
                'matches': group_round.matches
            }
        
        # Schedule knockout with gap
        knockout_slots = math.ceil(knockout_round.duration_minutes / self.config['time_slot_duration'])
        
        # Calculate gap slots (random between min and max)
        gap_minutes = np.random.randint(
            self.config['min_gap_between_rounds'],
            self.config['max_gap_between_rounds'] + 1
        )
        gap_slots = math.ceil(gap_minutes / self.config['time_slot_duration'])
        
        # Search start position
        if group_result:
            search_start = group_result['end_slot'] + gap_slots + 1
            preferred_day = self.time_grid[group_result['start_slot']].day
        else:
            search_start = 0
            preferred_day = None
        
        # Find slot for knockout
        slot_info = self.find_optimal_time_slot(
            knockout_slots, knockout_round.required_strips,
            knockout_round.weapon, knockout_round.required_referees,
            search_start, same_day=True, preferred_day=preferred_day
        )
        
        if not slot_info:
            # Rollback group allocation if knockout can't be scheduled
            if group_result:
                self.deallocate_time_slot(
                    group_result['start_slot'], group_slots,
                    group_round.weapon, group_result['strips'], group_round.required_referees
                )
            return None
        
        ko_start_slot, ko_strip_indices = slot_info
        
        # Allocate knockout stage
        self.allocate_time_slot(
            ko_start_slot, knockout_slots, f"{event_name}-knockout",
            knockout_round.weapon, ko_strip_indices, knockout_round.required_referees
        )
        
        knockout_result = {
            'start_slot': ko_start_slot,
            'end_slot': ko_start_slot + knockout_slots - 1,
            'strips': ko_strip_indices,
            'duration': knockout_round.duration_minutes,
            'matches': knockout_round.matches,
            'gap_minutes': gap_minutes
        }
        
        return {
            'event_name': event_name,
            'weapon': knockout_round.weapon,
            'participants': knockout_round.participants,
            'group': group_result,
            'knockout': knockout_result,
            'total_strips_used': max(
                len(group_result['strips']) if group_result else 0,
                len(knockout_result['strips'])
            )
        }
    
    def schedule_team_event(self, event_name: str, team_round: EventRound,
                           dependency: Dict) -> Optional[Dict]:
        """Schedule a team event after its dependency"""
        # Team events can be on next day
        dependency_end_slot = dependency['knockout']['end_slot']
        dependency_day = self.time_grid[dependency_end_slot].day
        
        team_slots = math.ceil(team_round.duration_minutes / self.config['time_slot_duration'])
        
        # Try same day first, then next day
        for day_offset in [0, 1]:
            target_day = dependency_day + day_offset
            if target_day > self.config['days']:
                continue
            
            # Find start of target day
            day_start_slot = next(
                (i for i, slot in enumerate(self.time_grid) if slot.day == target_day),
                dependency_end_slot + 1
            )
            
            slot_info = self.find_optimal_time_slot(
                team_slots, team_round.required_strips,
                team_round.weapon, team_round.required_referees,
                day_start_slot if day_offset > 0 else dependency_end_slot + 1,
                same_day=True, preferred_day=target_day
            )
            
            if slot_info:
                start_slot, strip_indices = slot_info
                
                # Allocate team event
                self.allocate_time_slot(
                    start_slot, team_slots, f"{event_name}-team",
                    team_round.weapon, strip_indices, team_round.required_referees
                )
                
                return {
                    'event_name': event_name,
                    'weapon': team_round.weapon,
                    'participants': team_round.participants,
                    'team': {
                        'start_slot': start_slot,
                        'end_slot': start_slot + team_slots - 1,
                        'strips': strip_indices,
                        'duration': team_round.duration_minutes,
                        'matches': team_round.matches
                    },
                    'dependency': dependency['event_name'],
                    'total_strips_used': len(strip_indices)
                }
        
        return None
    
    def generate_schedule(self) -> Dict:
        """Generate the complete optimized schedule"""
        # Calculate optimal referee distribution
        self.referee_distribution = self.optimize_referee_distribution()
        
        # Create event rounds
        self.event_rounds = self.create_event_rounds()
        
        # Schedule events
        scheduled, unscheduled = self.schedule_events()
        
        # Calculate statistics
        max_strips_used = 0
        for slot in self.time_grid:
            used_strips = sum(1 for s in slot.strip_assignments.values() if s is not None)
            max_strips_used = max(max_strips_used, used_strips)
        
        # Format output
        return self.format_schedule_output(scheduled, unscheduled, max_strips_used)
    
    def format_time(self, minutes: int) -> str:
        """Format minutes to HH:MM"""
        hours = minutes // 60
        mins = minutes % 60
        return f"{hours:02d}:{mins:02d}"
    
    def format_schedule_output(self, scheduled: Dict, unscheduled: List,
                              max_strips_used: int) -> Dict:
        """Format the schedule output with detailed information"""
        daily_schedule = defaultdict(list)
        
        # Process scheduled events
        for event_name, event_data in scheduled.items():
            # Add group stage if exists
            if event_data.get('group'):
                group = event_data['group']
                start_slot = self.time_grid[group['start_slot']]
                end_slot = self.time_grid[group['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Group Stage',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': group['duration'],
                    'strips': group['strips'],
                    'strip_count': len(group['strips']),
                    'matches': group['matches'],
                    'weapon': event_data['weapon']
                })
            
            # Add knockout stage
            if event_data.get('knockout'):
                knockout = event_data['knockout']
                start_slot = self.time_grid[knockout['start_slot']]
                end_slot = self.time_grid[knockout['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Knockout',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': knockout['duration'],
                    'strips': knockout['strips'],
                    'strip_count': len(knockout['strips']),
                    'matches': knockout['matches'],
                    'weapon': event_data['weapon'],
                    'gap_from_group': knockout.get('gap_minutes', 0)
                })
            
            # Add team event
            if event_data.get('team'):
                team = event_data['team']
                start_slot = self.time_grid[team['start_slot']]
                end_slot = self.time_grid[team['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Team Knockout',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': team['duration'],
                    'strips': team['strips'],
                    'strip_count': len(team['strips']),
                    'matches': team['matches'],
                    'weapon': event_data['weapon'],
                    'dependency': event_data.get('dependency')
                })
        
        # Sort events by start time within each day
        for day in daily_schedule:
            daily_schedule[day].sort(key=lambda x: x['start_time'])
        
        return {
            'configuration': self.config,
            'referee_distribution': self.referee_distribution,
            'total_referees': self.total_referees,
            'max_strips_used': max_strips_used,
            'strips_efficiency': f"{(max_strips_used / self.config['total_strips']) * 100:.1f}%",
            'total_events': len(self.events),
            'scheduled_events': len(scheduled),
            'unscheduled_events': len(unscheduled),
            'unscheduled_details': unscheduled,
            'daily_schedules': dict(daily_schedule),
            'scheduled_events_details': scheduled
        }

def print_optimized_schedule(schedule: Dict):
    """Print the optimized schedule in a clear format"""
    print("=" * 100)
    print("🤺 OPTIMIZED FENCING TOURNAMENT SCHEDULE")
    print("=" * 100)
    
    # Configuration
    config = schedule['configuration']
    print(f"\n📋 TOURNAMENT CONFIGURATION:")
    print(f"   Duration: {config['days']} days × {config['hours_per_day']} hours/day")
    print(f"   Total Strips: {config['total_strips']} (Maximum used: {schedule['max_strips_used']})")
    print(f"   Strip Efficiency: {schedule['strips_efficiency']}")
    print(f"   Knockout Percentage: {config['knockout_percentage']:.0%}")
    print(f"   Round Gap: {config['min_gap_between_rounds']}-{config['max_gap_between_rounds']} minutes")
    
    # Referee distribution
    ref_dist = schedule['referee_distribution']
    print(f"\n👨‍⚖️ REFEREE ALLOCATION (Total: {schedule['total_referees']}):")
    print(f"   Saber: {ref_dist['saber']} | Foil: {ref_dist['foil']} | Epee: {ref_dist['epee']}")
    
    # Summary
    print(f"\n📊 SCHEDULING SUMMARY:")
    print(f"   Total Events: {schedule['total_events']}")
    print(f"   Successfully Scheduled: {schedule['scheduled_events']}")
    print(f"   Failed to Schedule: {schedule['unscheduled_events']}")
    
    if schedule['unscheduled_details']:
        print(f"\n⚠️  UNSCHEDULED EVENTS:")
        for event in schedule['unscheduled_details']:
            print(f"   - {event['event']}: {event['reason']}")
            if 'dependency' in event:
                print(f"     (Dependency: {event['dependency']})")
    
    # Daily schedules
    for day, events in sorted(schedule['daily_schedules'].items()):
        print(f"\n{'='*100}")
        print(f"📅 DAY {day}")
        print(f"{'='*100}")
        
        if not events:
            print("   No events scheduled")
            continue
        
        # Group events by time for parallel visualization
        time_groups = defaultdict(list)
        for event in events:
            time_key = f"{event['start_time']}-{event['end_time']}"
            time_groups[time_key].append(event)
        
        for time_range, concurrent_events in sorted(time_groups.items()):
            if len(concurrent_events) == 1:
                event = concurrent_events[0]
                weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                emoji = weapon_emoji.get(event['weapon'], '🤺')
                
                print(f"\n   {emoji} {event['start_time']} - {event['end_time']} | {event['event_name']} - {event['round_type']}")
                print(f"      Strips: {event['strip_count']} (#{', #'.join(map(str, event['strips']))})")
                print(f"      Matches: {event['matches']} | Duration: {event['duration_minutes']} min")
                
                if event['round_type'] == 'Knockout' and event.get('gap_from_group'):
                    print(f"      Gap from group stage: {event['gap_from_group']} minutes")
                elif event['round_type'] == 'Team Knockout' and event.get('dependency'):
                    print(f"      After: {event['dependency']}")
            else:
                print(f"\n   🔄 {time_range} | {len(concurrent_events)} PARALLEL EVENTS:")
                for event in concurrent_events:
                    weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                    emoji = weapon_emoji.get(event['weapon'], '🤺')
                    
                    print(f"      {emoji} {event['event_name']} - {event['round_type']}")
                    print(f"         Strips: #{', #'.join(map(str, event['strips']))}")
                    print(f"         Matches: {event['matches']} | Duration: {event['duration_minutes']} min")

# Helper function to run the optimized scheduler
def run_optimized_scheduler(events_data=None, config=None):
    """Run the optimized tournament scheduler"""
    if events_data is None:
        # Use sample data
        events_data = [
            {'name': 'U14女子重剑个人', 'participants': 63},
            {'name': 'U14女子佩剑个人', 'participants': 61},
            {'name': 'U16男子花剑个人', 'participants': 39},
            {'name': 'U12男子花剑个人', 'participants': 224},
            {'name': 'U10男子佩剑个人', 'participants': 80},
            {'name': 'U14女子重剑团体', 'participants': 5},
            {'name': 'U14女子佩剑团体', 'participants': 6}
        ]
    
    # Create scheduler
    scheduler = OptimizedFencingScheduler(events_data, config)
    
    # Generate schedule
    schedule = scheduler.generate_schedule()
    
    # Print results
    print_optimized_schedule(schedule)
    
    return schedule

In [28]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

@dataclass
class TimeSlot:
    """Represents a time slot with strip assignments"""
    start_time: int
    end_time: int
    day: int
    strip_assignments: Dict[int, Optional[str]]  # strip_id -> event_name
    referee_usage: Dict[str, int]  # weapon -> count
    
    def available_strips(self) -> List[int]:
        """Get list of available strip indices"""
        return [idx for idx, event in self.strip_assignments.items() if event is None]
    
    def assign_strips(self, event_name: str, strip_indices: List[int]) -> bool:
        """Assign specific strips to an event"""
        # Check if all requested strips are available
        for idx in strip_indices:
            if self.strip_assignments.get(idx) is not None:
                return False
        
        # Assign strips
        for idx in strip_indices:
            self.strip_assignments[idx] = event_name
        return True
    
    def release_strips(self, strip_indices: List[int]):
        """Release strips"""
        for idx in strip_indices:
            self.strip_assignments[idx] = None

@dataclass
class EventRound:
    """Represents a round of an event"""
    event_name: str
    round_type: str  # 'group', 'knockout', 'team'
    weapon: str
    participants: int
    matches: int
    duration_minutes: int
    required_strips: int
    required_referees: int
    priority: int  # Lower is higher priority

class OptimizedFencingScheduler:
    def __init__(self, events_data, config=None):
        """Initialize the optimized fencing tournament scheduler."""
        if isinstance(events_data, pd.DataFrame):
            self.events = events_data.to_dict('records')
        else:
            self.events = events_data
            
        # Default configuration
        default_config = {
            'days': 4,
            'hours_per_day': 12,
            'knockout_percentage': 0.7,
            'total_strips': 40,
            'min_gap_between_rounds': 30,  # minutes
            'max_gap_between_rounds': 60,  # minutes
            'max_round_duration': 180,  # minutes (3 hours)
            'time_slot_duration': 5,   # minutes (smaller for better granularity)
            'referee_multiplier': 1.2
        }
        
        if config:
            default_config.update(config)
        self.config = default_config
        
        # Match durations by weapon (in minutes)
        self.match_durations = {
            'saber': {'group': 5, 'knockout': 10},
            'foil': {'group': 8, 'knockout': 16},
            'epee': {'group': 8, 'knockout': 16}
        }
        
        self.total_referees = int(self.config['total_strips'] * self.config['referee_multiplier'])
        
        # Calculate total time slots
        total_minutes = self.config['days'] * self.config['hours_per_day'] * 60
        self.total_time_slots = total_minutes // self.config['time_slot_duration']
        
        # Initialize time grid
        self.time_grid = self._create_time_grid()
        
        # Track scheduled events
        self.scheduled_events = {}
        self.event_rounds = []
        
    def _create_time_grid(self) -> List[TimeSlot]:
        """Create optimized time slot grid with strip tracking"""
        grid = []
        for slot_idx in range(self.total_time_slots):
            start_time = slot_idx * self.config['time_slot_duration']
            end_time = start_time + self.config['time_slot_duration']
            day = (start_time // (self.config['hours_per_day'] * 60)) + 1
            
            # Initialize all strips as available
            strip_assignments = {i: None for i in range(self.config['total_strips'])}
            
            grid.append(TimeSlot(
                start_time=start_time,
                end_time=end_time,
                day=day,
                strip_assignments=strip_assignments,
                referee_usage={'saber': 0, 'foil': 0, 'epee': 0}
            ))
        return grid
    
    def get_weapon_type(self, event_name: str) -> str:
        """Determine weapon type from event name."""
        if '佩剑' in event_name:
            return 'saber'
        elif '花剑' in event_name:
            return 'foil'
        elif '重剑' in event_name:
            return 'epee'
        return 'foil'  # default
    
    def is_team_event(self, event_name: str) -> bool:
        """Check if event is a team event."""
        return '团体' in event_name
    
    def calculate_optimal_groups(self, participants: int) -> Tuple[int, List[int]]:
        """Calculate optimal group configuration to minimize variance"""
        if participants <= 7:
            return 1, [participants]
        
        # Try to make groups of 6 or 7 with minimal variance
        best_config = None
        min_variance = float('inf')
        
        for target_size in [6, 7]:
            num_groups = participants // target_size
            remainder = participants % target_size
            
            if remainder == 0:
                return num_groups, [target_size] * num_groups
            
            # Distribute remainder evenly
            group_sizes = [target_size] * num_groups
            for i in range(remainder):
                group_sizes[i % num_groups] += 1
            
            # Calculate variance
            variance = np.var(group_sizes)
            if variance < min_variance:
                min_variance = variance
                best_config = (len(group_sizes), group_sizes)
        
        return best_config
    
    def calculate_group_matches(self, participants: int) -> int:
        """Calculate total matches in group stage."""
        _, group_sizes = self.calculate_optimal_groups(participants)
        total_matches = sum(size * (size - 1) // 2 for size in group_sizes)
        return total_matches
    
    def calculate_knockout_structure(self, participants: int, percentage: float) -> Dict:
        """Calculate optimized knockout structure"""
        knockout_participants = int(participants * percentage)
        
        # Find the appropriate power of 2
        power_of_2 = 1
        while power_of_2 * 2 <= knockout_participants:
            power_of_2 *= 2
        
        # Calculate preliminary matches needed
        if knockout_participants > power_of_2:
            # Only bottom ranked fencers need preliminary
            preliminary_participants = 2 * (knockout_participants - power_of_2)
            preliminary_matches = knockout_participants - power_of_2
        else:
            preliminary_participants = 0
            preliminary_matches = 0
        
        # Main knockout rounds
        knockout_rounds = []
        remaining = power_of_2
        round_num = 1
        
        while remaining > 1:
            matches = remaining // 2
            knockout_rounds.append({
                'round': round_num,
                'matches': matches,
                'participants': remaining
            })
            remaining //= 2
            round_num += 1
        
        return {
            'total_participants': knockout_participants,
            'preliminary_matches': preliminary_matches,
            'preliminary_participants': preliminary_participants,
            'knockout_rounds': knockout_rounds,
            'total_knockout_matches': sum(r['matches'] for r in knockout_rounds),
            'total_matches': preliminary_matches + sum(r['matches'] for r in knockout_rounds)
        }
    
    def optimize_referee_distribution(self) -> Dict[str, int]:
        """Optimize referee distribution based on actual match load"""
        weapon_time_requirements = {'saber': 0, 'foil': 0, 'epee': 0}
        
        for event in self.events:
            weapon = self.get_weapon_type(event['name'])
            
            if self.is_team_event(event['name']):
                # Team events - single elimination
                matches = event['participants'] - 1
                duration = self.match_durations[weapon]['knockout']
                weapon_time_requirements[weapon] += matches * duration
            else:
                # Individual events
                # Group stage
                group_matches = self.calculate_group_matches(event['participants'])
                group_duration = self.match_durations[weapon]['group']
                weapon_time_requirements[weapon] += group_matches * group_duration
                
                # Knockout stage
                knockout_data = self.calculate_knockout_structure(
                    event['participants'], self.config['knockout_percentage']
                )
                knockout_duration = self.match_durations[weapon]['knockout']
                weapon_time_requirements[weapon] += knockout_data['total_matches'] * knockout_duration
        
        # Distribute referees proportionally with minimum guarantee
        total_time = sum(weapon_time_requirements.values())
        if total_time == 0:
            return {'saber': 10, 'foil': 10, 'epee': 10}
        
        referee_distribution = {}
        allocated = 0
        
        for weapon in ['saber', 'foil', 'epee']:
            if weapon_time_requirements[weapon] > 0:
                # Proportional allocation with minimum of 3 referees
                ideal_refs = (weapon_time_requirements[weapon] / total_time) * self.total_referees
                referee_distribution[weapon] = max(3, round(ideal_refs))
                allocated += referee_distribution[weapon]
            else:
                referee_distribution[weapon] = 0
        
        # Adjust if over-allocated
        while allocated > self.total_referees:
            # Reduce from weapon with most referees
            max_weapon = max(referee_distribution.keys(), key=lambda k: referee_distribution[k])
            if referee_distribution[max_weapon] > 3:
                referee_distribution[max_weapon] -= 1
                allocated -= 1
        
        # Distribute any remaining referees
        while allocated < self.total_referees:
            # Add to weapon with highest time requirement per referee
            best_weapon = max(
                [w for w in weapon_time_requirements if referee_distribution[w] > 0],
                key=lambda w: weapon_time_requirements[w] / referee_distribution[w]
            )
            referee_distribution[best_weapon] += 1
            allocated += 1
        
        return referee_distribution
    
    def calculate_round_requirements(self, matches: int, weapon: str, round_type: str,
                                   available_referees: int) -> Dict:
        """Calculate optimized round requirements"""
        if matches == 0:
            return {
                'strips_needed': 0,
                'duration': 0,
                'concurrent_matches': 0,
                'rounds': 0,
                'valid': True
            }
        
        match_duration = self.match_durations[weapon]['group' if round_type == 'group' else 'knockout']
        
        # Maximum concurrent matches limited by referees
        max_concurrent = min(matches, available_referees, self.config['total_strips'])
        
        # Optimize for minimum strips while respecting 3-hour constraint
        best_config = None
        min_strips = float('inf')
        
        for strips in range(1, max_concurrent + 1):
            rounds_needed = math.ceil(matches / strips)
            total_duration = rounds_needed * match_duration
            
            if total_duration <= self.config['max_round_duration']:
                if strips < min_strips:
                    min_strips = strips
                    best_config = {
                        'strips_needed': strips,
                        'duration': total_duration,
                        'concurrent_matches': strips,
                        'rounds': rounds_needed,
                        'valid': True
                    }
        
        if best_config is None:
            # If no valid config found, use maximum strips
            best_config = {
                'strips_needed': max_concurrent,
                'duration': math.ceil(matches / max_concurrent) * match_duration,
                'concurrent_matches': max_concurrent,
                'rounds': math.ceil(matches / max_concurrent),
                'valid': False
            }
        
        return best_config
    
    def find_optimal_time_slot(self, duration_slots: int, required_strips: int, 
                             weapon: str, required_referees: int, 
                             start_search: int = 0, same_day: bool = True,
                             preferred_day: Optional[int] = None) -> Optional[Tuple[int, List[int]]]:
        """Find optimal time slot and strip assignment"""
        current_day = self.time_grid[start_search].day if start_search < len(self.time_grid) else None
        
        for start_slot in range(start_search, len(self.time_grid) - duration_slots + 1):
            slot = self.time_grid[start_slot]
            
            # Check day constraint
            if same_day and current_day and slot.day != current_day:
                continue
            if preferred_day and slot.day != preferred_day:
                continue
            
            # Check if all slots in duration are in same day
            end_slot_idx = start_slot + duration_slots - 1
            if end_slot_idx >= len(self.time_grid):
                continue
            
            end_day = self.time_grid[end_slot_idx].day
            if slot.day != end_day:
                continue
            
            # Find contiguous strips available for entire duration
            available_strips_sets = []
            valid_allocation = True
            
            for slot_idx in range(start_slot, start_slot + duration_slots):
                current_slot = self.time_grid[slot_idx]
                
                # Check referee availability
                if current_slot.referee_usage.get(weapon, 0) + required_referees > \
                   self.referee_distribution.get(weapon, 0):
                    valid_allocation = False
                    break
                
                available_strips = current_slot.available_strips()
                if len(available_strips) < required_strips:
                    valid_allocation = False
                    break
                
                available_strips_sets.append(set(available_strips))
            
            if not valid_allocation:
                continue
            
            # Find strips available for entire duration
            common_strips = set.intersection(*available_strips_sets)
            if len(common_strips) >= required_strips:
                # Select lowest numbered strips for consistency
                selected_strips = sorted(list(common_strips))[:required_strips]
                return start_slot, selected_strips
        
        return None
    
    def allocate_time_slot(self, start_slot: int, duration_slots: int, 
                          event_name: str, weapon: str, strip_indices: List[int],
                          required_referees: int):
        """Allocate time slots and strips for an event"""
        for slot_idx in range(start_slot, start_slot + duration_slots):
            slot = self.time_grid[slot_idx]
            slot.assign_strips(event_name, strip_indices)
            slot.referee_usage[weapon] = slot.referee_usage.get(weapon, 0) + required_referees
    
    def deallocate_time_slot(self, start_slot: int, duration_slots: int,
                            weapon: str, strip_indices: List[int], required_referees: int):
        """Deallocate time slots and strips"""
        for slot_idx in range(start_slot, start_slot + duration_slots):
            slot = self.time_grid[slot_idx]
            slot.release_strips(strip_indices)
            slot.referee_usage[weapon] = max(0, slot.referee_usage.get(weapon, 0) - required_referees)
    
    def create_event_rounds(self) -> List[EventRound]:
        """Create all event rounds with priorities"""
        rounds = []
        
        # Process events by priority: individual events first, then team events
        individual_events = [e for e in self.events if not self.is_team_event(e['name'])]
        team_events = [e for e in self.events if self.is_team_event(e['name'])]
        
        # Sort individual events by participant count (larger events first for better packing)
        individual_events.sort(key=lambda x: -x['participants'])
        
        # Create rounds for individual events
        for idx, event in enumerate(individual_events):
            weapon = self.get_weapon_type(event['name'])
            
            # Group stage
            group_matches = self.calculate_group_matches(event['participants'])
            if group_matches > 0:
                group_req = self.calculate_round_requirements(
                    group_matches, weapon, 'group',
                    self.referee_distribution.get(weapon, 0)
                )
                
                rounds.append(EventRound(
                    event_name=event['name'],
                    round_type='group',
                    weapon=weapon,
                    participants=event['participants'],
                    matches=group_matches,
                    duration_minutes=group_req['duration'],
                    required_strips=group_req['strips_needed'],
                    required_referees=group_req['concurrent_matches'],
                    priority=idx * 2  # Even priorities for group stages
                ))
            
            # Knockout stage
            knockout_data = self.calculate_knockout_structure(
                event['participants'], self.config['knockout_percentage']
            )
            
            knockout_req = self.calculate_round_requirements(
                knockout_data['total_matches'], weapon, 'knockout',
                self.referee_distribution.get(weapon, 0)
            )
            
            rounds.append(EventRound(
                event_name=event['name'],
                round_type='knockout',
                weapon=weapon,
                participants=knockout_data['total_participants'],
                matches=knockout_data['total_matches'],
                duration_minutes=knockout_req['duration'],
                required_strips=knockout_req['strips_needed'],
                required_referees=knockout_req['concurrent_matches'],
                priority=idx * 2 + 1  # Odd priorities for knockout stages
            ))
        
        # Create rounds for team events
        for idx, event in enumerate(team_events):
            weapon = self.get_weapon_type(event['name'])
            matches = event['participants'] - 1
            
            team_req = self.calculate_round_requirements(
                matches, weapon, 'knockout',
                self.referee_distribution.get(weapon, 0)
            )
            
            rounds.append(EventRound(
                event_name=event['name'],
                round_type='team',
                weapon=weapon,
                participants=event['participants'],
                matches=matches,
                duration_minutes=team_req['duration'],
                required_strips=team_req['strips_needed'],
                required_referees=team_req['concurrent_matches'],
                priority=1000 + idx  # High priority number for team events
            ))
        
        return rounds
    
    def schedule_events(self) -> Tuple[Dict, List[Dict]]:
        """Schedule all events using constraint programming approach"""
        scheduled = {}
        unscheduled = []
        
        # Group rounds by event
        event_rounds_map = defaultdict(list)
        for round_obj in self.event_rounds:
            event_rounds_map[round_obj.event_name].append(round_obj)
        
        # Schedule individual events
        for event_name, rounds in event_rounds_map.items():
            if any(r.round_type == 'team' for r in rounds):
                continue  # Skip team events for now
            
            event_scheduled = self.schedule_individual_event(event_name, rounds)
            if event_scheduled:
                scheduled[event_name] = event_scheduled
            else:
                unscheduled.append({
                    'event': event_name,
                    'reason': 'Could not find suitable time slots',
                    'rounds': [r.round_type for r in rounds]
                })
        
        # Schedule team events after their dependencies
        for event_name, rounds in event_rounds_map.items():
            if not any(r.round_type == 'team' for r in rounds):
                continue
            
            # Find dependency (individual event)
            individual_event = event_name.replace('团体', '个人')
            if individual_event in scheduled:
                dependency = scheduled[individual_event]
                team_scheduled = self.schedule_team_event(
                    event_name, rounds[0], dependency
                )
                if team_scheduled:
                    scheduled[event_name] = team_scheduled
                else:
                    unscheduled.append({
                        'event': event_name,
                        'reason': 'Could not schedule after dependency',
                        'dependency': individual_event
                    })
            else:
                unscheduled.append({
                    'event': event_name,
                    'reason': 'Dependency not scheduled',
                    'dependency': individual_event
                })
        
        return schedule

# Comparison function to test different configurations
def compare_configurations(events_data, configs_to_test):
    """Compare different configurations to find optimal settings"""
    results = []
    
    print("🔬 CONFIGURATION COMPARISON")
    print("=" * 80)
    
    for config_name, config in configs_to_test.items():
        print(f"\n📋 Testing: {config_name}")
        print(f"   Days: {config['days']}, Hours/day: {config['hours_per_day']}, Strips: {config['total_strips']}")
        
        scheduler = OptimizedFencingScheduler(events_data, config)
        schedule = scheduler.generate_schedule()
        
        results.append({
            'name': config_name,
            'config': config,
            'max_strips_used': schedule['max_strips_used'],
            'efficiency': schedule['strips_efficiency'],
            'scheduled': schedule['scheduled_events'],
            'unscheduled': schedule['unscheduled_events'],
            'success_rate': f"{(schedule['scheduled_events'] / schedule['total_events']) * 100:.1f}%"
        })
        
        print(f"   ✅ Scheduled: {schedule['scheduled_events']}/{schedule['total_events']}")
        print(f"   📊 Max strips used: {schedule['max_strips_used']}")
        print(f"   💡 Efficiency: {schedule['strips_efficiency']}")
    
    # Find best configuration
    best = max(results, key=lambda x: (x['scheduled'], -x['max_strips_used']))
    print(f"\n🏆 BEST CONFIGURATION: {best['name']}")
    print(f"   Success rate: {best['success_rate']}")
    print(f"   Strip efficiency: {best['efficiency']}")
    
    return results

# Function to load tournament data from Excel (same format as original)
def load_tournament_data_optimized(file_path):
    """Load tournament data from Excel file"""
    df = pd.read_excel(file_path)
    events = []
    for _, row in df.iterrows():
        if pd.notna(row.iloc[1]) and pd.notna(row.iloc[3]) and row.iloc[1] != '组别':
            events.append({
                'name': row.iloc[1],
                'participants': int(row.iloc[3])
            })
    return events

# Main function to run from file or sample data
def main(file_path=None, use_sample=True, custom_config=None):
    """Main function to run the optimized scheduler
    
    Args:
        file_path: Path to Excel file with tournament data
        use_sample: If True and no file_path, use sample data
        custom_config: Custom configuration dictionary
    """
    # Load events
    if file_path:
        events = load_tournament_data_optimized(file_path)
        print(f"📁 Loaded {len(events)} events from {file_path}")
    elif use_sample:
        events = [
            {'name': 'U12男子花剑个人', 'participants': 224},
            {'name': 'U12女子重剑个人', 'participants': 118},
            {'name': 'U10男子花剑个人', 'participants': 139},
            {'name': 'U14女子重剑个人', 'participants': 63},
            {'name': 'U14女子佩剑个人', 'participants': 61},
            {'name': 'U16男子花剑个人', 'participants': 39},
            {'name': 'U10男子佩剑个人', 'participants': 80},
            {'name': 'U12男子花剑团体', 'participants': 8},
            {'name': 'U14女子重剑团体', 'participants': 5},
            {'name': 'U14女子佩剑团体', 'participants': 6}
        ]
        print(f"📊 Using {len(events)} sample events")
    else:
        print("❌ No data source specified")
        return None
    
    # Default optimized configuration
    default_config = {
        'days': 2,
        'hours_per_day': 10,
        'knockout_percentage': 0.7,
        'total_strips': 20,
        'min_gap_between_rounds': 30,
        'max_gap_between_rounds': 60,
        'max_round_duration': 180,
        'time_slot_duration': 5,
        'referee_multiplier': 1.2
    }
    
    if custom_config:
        default_config.update(custom_config)
    
    # Run scheduler
    return run_optimized_scheduler(events, default_config)

# Example usage
if __name__ == "__main__":
    # Example 1: Run with sample data
    print("Example 1: Running with sample data")
    schedule = main(use_sample=True)
    
    # Example 2: Test different configurations
    print("\n\nExample 2: Testing different configurations")
    test_events = [
        {'name': 'U12男子花剑个人', 'participants': 224},
        {'name': 'U12女子重剑个人', 'participants': 118},
        {'name': 'U12男子花剑团体', 'participants': 8}
    ]
    
    configs = {
        'Minimal': {'days': 2, 'hours_per_day': 8, 'total_strips': 15},
        'Standard': {'days': 2, 'hours_per_day': 10, 'total_strips': 20},
        'Extended': {'days': 3, 'hours_per_day': 8, 'total_strips': 18}
    }
    
    compare_configurations(test_events, configs)
    
    def schedule_individual_event(self, event_name: str, rounds: List[EventRound]) -> Optional[Dict]:
        """Schedule an individual event (group + knockout)"""
        group_round = next((r for r in rounds if r.round_type == 'group'), None)
        knockout_round = next((r for r in rounds if r.round_type == 'knockout'), None)
        
        if not knockout_round:
            return None
        
        # Calculate slot requirements
        group_slots = 0
        group_result = None
        
        if group_round:
            group_slots = math.ceil(group_round.duration_minutes / self.config['time_slot_duration'])
            
            # Find slot for group stage
            slot_info = self.find_optimal_time_slot(
                group_slots, group_round.required_strips,
                group_round.weapon, group_round.required_referees, 0
            )
            
            if not slot_info:
                return None
            
            start_slot, strip_indices = slot_info
            
            # Allocate group stage
            self.allocate_time_slot(
                start_slot, group_slots, f"{event_name}-group",
                group_round.weapon, strip_indices, group_round.required_referees
            )
            
            group_result = {
                'start_slot': start_slot,
                'end_slot': start_slot + group_slots - 1,
                'strips': strip_indices,
                'duration': group_round.duration_minutes,
                'matches': group_round.matches
            }
        
        # Schedule knockout with gap
        knockout_slots = math.ceil(knockout_round.duration_minutes / self.config['time_slot_duration'])
        
        # Calculate gap slots (random between min and max)
        gap_minutes = np.random.randint(
            self.config['min_gap_between_rounds'],
            self.config['max_gap_between_rounds'] + 1
        )
        gap_slots = math.ceil(gap_minutes / self.config['time_slot_duration'])
        
        # Search start position
        if group_result:
            search_start = group_result['end_slot'] + gap_slots + 1
            preferred_day = self.time_grid[group_result['start_slot']].day
        else:
            search_start = 0
            preferred_day = None
        
        # Find slot for knockout
        slot_info = self.find_optimal_time_slot(
            knockout_slots, knockout_round.required_strips,
            knockout_round.weapon, knockout_round.required_referees,
            search_start, same_day=True, preferred_day=preferred_day
        )
        
        if not slot_info:
            # Rollback group allocation if knockout can't be scheduled
            if group_result:
                self.deallocate_time_slot(
                    group_result['start_slot'], group_slots,
                    group_round.weapon, group_result['strips'], group_round.required_referees
                )
            return None
        
        ko_start_slot, ko_strip_indices = slot_info
        
        # Allocate knockout stage
        self.allocate_time_slot(
            ko_start_slot, knockout_slots, f"{event_name}-knockout",
            knockout_round.weapon, ko_strip_indices, knockout_round.required_referees
        )
        
        knockout_result = {
            'start_slot': ko_start_slot,
            'end_slot': ko_start_slot + knockout_slots - 1,
            'strips': ko_strip_indices,
            'duration': knockout_round.duration_minutes,
            'matches': knockout_round.matches,
            'gap_minutes': gap_minutes
        }
        
        return {
            'event_name': event_name,
            'weapon': knockout_round.weapon,
            'participants': knockout_round.participants,
            'group': group_result,
            'knockout': knockout_result,
            'total_strips_used': max(
                len(group_result['strips']) if group_result else 0,
                len(knockout_result['strips'])
            )
        }
    
    def schedule_team_event(self, event_name: str, team_round: EventRound,
                           dependency: Dict) -> Optional[Dict]:
        """Schedule a team event after its dependency"""
        # Team events can be on next day
        dependency_end_slot = dependency['knockout']['end_slot']
        dependency_day = self.time_grid[dependency_end_slot].day
        
        team_slots = math.ceil(team_round.duration_minutes / self.config['time_slot_duration'])
        
        # Try same day first, then next day
        for day_offset in [0, 1]:
            target_day = dependency_day + day_offset
            if target_day > self.config['days']:
                continue
            
            # Find start of target day
            day_start_slot = next(
                (i for i, slot in enumerate(self.time_grid) if slot.day == target_day),
                dependency_end_slot + 1
            )
            
            slot_info = self.find_optimal_time_slot(
                team_slots, team_round.required_strips,
                team_round.weapon, team_round.required_referees,
                day_start_slot if day_offset > 0 else dependency_end_slot + 1,
                same_day=True, preferred_day=target_day
            )
            
            if slot_info:
                start_slot, strip_indices = slot_info
                
                # Allocate team event
                self.allocate_time_slot(
                    start_slot, team_slots, f"{event_name}-team",
                    team_round.weapon, strip_indices, team_round.required_referees
                )
                
                return {
                    'event_name': event_name,
                    'weapon': team_round.weapon,
                    'participants': team_round.participants,
                    'team': {
                        'start_slot': start_slot,
                        'end_slot': start_slot + team_slots - 1,
                        'strips': strip_indices,
                        'duration': team_round.duration_minutes,
                        'matches': team_round.matches
                    },
                    'dependency': dependency['event_name'],
                    'total_strips_used': len(strip_indices)
                }
        
        return None
    
    def generate_schedule(self) -> Dict:
        """Generate the complete optimized schedule"""
        # Calculate optimal referee distribution
        self.referee_distribution = self.optimize_referee_distribution()
        
        # Create event rounds
        self.event_rounds = self.create_event_rounds()
        
        # Schedule events
        scheduled, unscheduled = self.schedule_events()
        
        # Calculate statistics
        max_strips_used = 0
        for slot in self.time_grid:
            used_strips = sum(1 for s in slot.strip_assignments.values() if s is not None)
            max_strips_used = max(max_strips_used, used_strips)
        
        # Format output
        return self.format_schedule_output(scheduled, unscheduled, max_strips_used)
    
    def format_time(self, minutes: int) -> str:
        """Format minutes to HH:MM"""
        hours = minutes // 60
        mins = minutes % 60
        return f"{hours:02d}:{mins:02d}"
    
    def format_schedule_output(self, scheduled: Dict, unscheduled: List,
                              max_strips_used: int) -> Dict:
        """Format the schedule output with detailed information"""
        daily_schedule = defaultdict(list)
        
        # Process scheduled events
        for event_name, event_data in scheduled.items():
            # Add group stage if exists
            if event_data.get('group'):
                group = event_data['group']
                start_slot = self.time_grid[group['start_slot']]
                end_slot = self.time_grid[group['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Group Stage',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': group['duration'],
                    'strips': group['strips'],
                    'strip_count': len(group['strips']),
                    'matches': group['matches'],
                    'weapon': event_data['weapon']
                })
            
            # Add knockout stage
            if event_data.get('knockout'):
                knockout = event_data['knockout']
                start_slot = self.time_grid[knockout['start_slot']]
                end_slot = self.time_grid[knockout['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Knockout',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': knockout['duration'],
                    'strips': knockout['strips'],
                    'strip_count': len(knockout['strips']),
                    'matches': knockout['matches'],
                    'weapon': event_data['weapon'],
                    'gap_from_group': knockout.get('gap_minutes', 0)
                })
            
            # Add team event
            if event_data.get('team'):
                team = event_data['team']
                start_slot = self.time_grid[team['start_slot']]
                end_slot = self.time_grid[team['end_slot']]
                
                daily_schedule[start_slot.day].append({
                    'event_name': event_name,
                    'round_type': 'Team Knockout',
                    'start_time': self.format_time(start_slot.start_time),
                    'end_time': self.format_time(end_slot.end_time),
                    'duration_minutes': team['duration'],
                    'strips': team['strips'],
                    'strip_count': len(team['strips']),
                    'matches': team['matches'],
                    'weapon': event_data['weapon'],
                    'dependency': event_data.get('dependency')
                })
        
        # Sort events by start time within each day
        for day in daily_schedule:
            daily_schedule[day].sort(key=lambda x: x['start_time'])
        
        return {
            'configuration': self.config,
            'referee_distribution': self.referee_distribution,
            'total_referees': self.total_referees,
            'max_strips_used': max_strips_used,
            'strips_efficiency': f"{(max_strips_used / self.config['total_strips']) * 100:.1f}%",
            'total_events': len(self.events),
            'scheduled_events': len(scheduled),
            'unscheduled_events': len(unscheduled),
            'unscheduled_details': unscheduled,
            'daily_schedules': dict(daily_schedule),
            'scheduled_events_details': scheduled
        }

def print_optimized_schedule(schedule: Dict):
    """Print the optimized schedule in a clear format"""
    print("=" * 100)
    print("🤺 OPTIMIZED FENCING TOURNAMENT SCHEDULE")
    print("=" * 100)
    
    # Configuration
    config = schedule['configuration']
    print(f"\n📋 TOURNAMENT CONFIGURATION:")
    print(f"   Duration: {config['days']} days × {config['hours_per_day']} hours/day")
    print(f"   Total Strips: {config['total_strips']} (Maximum used: {schedule['max_strips_used']})")
    print(f"   Strip Efficiency: {schedule['strips_efficiency']}")
    print(f"   Knockout Percentage: {config['knockout_percentage']:.0%}")
    print(f"   Round Gap: {config['min_gap_between_rounds']}-{config['max_gap_between_rounds']} minutes")
    
    # Referee distribution
    ref_dist = schedule['referee_distribution']
    print(f"\n👨‍⚖️ REFEREE ALLOCATION (Total: {schedule['total_referees']}):")
    print(f"   Saber: {ref_dist['saber']} | Foil: {ref_dist['foil']} | Epee: {ref_dist['epee']}")
    
    # Summary
    print(f"\n📊 SCHEDULING SUMMARY:")
    print(f"   Total Events: {schedule['total_events']}")
    print(f"   Successfully Scheduled: {schedule['scheduled_events']}")
    print(f"   Failed to Schedule: {schedule['unscheduled_events']}")
    
    if schedule['unscheduled_details']:
        print(f"\n⚠️  UNSCHEDULED EVENTS:")
        for event in schedule['unscheduled_details']:
            print(f"   - {event['event']}: {event['reason']}")
            if 'dependency' in event:
                print(f"     (Dependency: {event['dependency']})")
    
    # Daily schedules
    for day, events in sorted(schedule['daily_schedules'].items()):
        print(f"\n{'='*100}")
        print(f"📅 DAY {day}")
        print(f"{'='*100}")
        
        if not events:
            print("   No events scheduled")
            continue
        
        # Group events by time for parallel visualization
        time_groups = defaultdict(list)
        for event in events:
            time_key = f"{event['start_time']}-{event['end_time']}"
            time_groups[time_key].append(event)
        
        for time_range, concurrent_events in sorted(time_groups.items()):
            if len(concurrent_events) == 1:
                event = concurrent_events[0]
                weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                emoji = weapon_emoji.get(event['weapon'], '🤺')
                
                print(f"\n   {emoji} {event['start_time']} - {event['end_time']} | {event['event_name']} - {event['round_type']}")
                print(f"      Strips: {event['strip_count']} (#{', #'.join(map(str, event['strips']))})")
                print(f"      Matches: {event['matches']} | Duration: {event['duration_minutes']} min")
                
                if event['round_type'] == 'Knockout' and event.get('gap_from_group'):
                    print(f"      Gap from group stage: {event['gap_from_group']} minutes")
                elif event['round_type'] == 'Team Knockout' and event.get('dependency'):
                    print(f"      After: {event['dependency']}")
            else:
                print(f"\n   🔄 {time_range} | {len(concurrent_events)} PARALLEL EVENTS:")
                for event in concurrent_events:
                    weapon_emoji = {'saber': '⚔️', 'foil': '🗡️', 'epee': '🤺'}
                    emoji = weapon_emoji.get(event['weapon'], '🤺')
                    
                    print(f"      {emoji} {event['event_name']} - {event['round_type']}")
                    print(f"         Strips: #{', #'.join(map(str, event['strips']))}")
                    print(f"         Matches: {event['matches']} | Duration: {event['duration_minutes']} min")

# Helper function to run the optimized scheduler
def run_optimized_scheduler(events_data=None, config=None):
    """Run the optimized tournament scheduler"""
    if events_data is None:
        # Use sample data
        events_data = [
            {'name': 'U14女子重剑个人', 'participants': 63},
            {'name': 'U14女子佩剑个人', 'participants': 61},
            {'name': 'U16男子花剑个人', 'participants': 39},
            {'name': 'U12男子花剑个人', 'participants': 224},
            {'name': 'U10男子佩剑个人', 'participants': 80},
            {'name': 'U14女子重剑团体', 'participants': 5},
            {'name': 'U14女子佩剑团体', 'participants': 6}
        ]
    
    # Create scheduler
    scheduler = OptimizedFencingScheduler(events_data, config)
    
    # Generate schedule
    schedule = scheduler.generate_schedule()
    
    # Print results
    print_optimized_schedule(schedule)
    
    return schedule

# Demonstration function showing improvements
def demonstrate_optimization(events_data=None):
    """Demonstrate the optimization improvements"""
    if events_data is None:
        events_data = [
            {'name': 'U12男子花剑个人', 'participants': 224},
            {'name': 'U12女子重剑个人', 'participants': 118},
            {'name': 'U10男子花剑个人', 'participants': 139},
            {'name': 'U14女子重剑个人', 'participants': 63},
            {'name': 'U14女子佩剑个人', 'participants': 61},
            {'name': 'U16男子花剑个人', 'participants': 39},
            {'name': 'U10男子佩剑个人', 'participants': 80},
            {'name': 'U12男子花剑团体', 'participants': 8},
            {'name': 'U14女子重剑团体', 'participants': 5},
            {'name': 'U14女子佩剑团体', 'participants': 6}
        ]
    
    print("🎯 OPTIMIZATION DEMONSTRATION")
    print("=" * 80)
    
    # Test with minimal resources
    config = {
        'days': 2,
        'hours_per_day': 10,
        'knockout_percentage': 0.7,
        'total_strips': 18,  # Limited strips to test optimization
        'min_gap_between_rounds': 30,
        'max_gap_between_rounds': 60,
        'max_round_duration': 180,
        'time_slot_duration': 5
    }
    
    print(f"\n📊 Testing with {len(events_data)} events on {config['total_strips']} strips")
    print(f"   Total participants: {sum(e['participants'] for e in events_data)}")
    
    # Run optimized scheduler
    schedule = run_optimized_scheduler(events_data, config)
    
    print("\n🔍 KEY OPTIMIZATION METRICS:")
    print(f"   Strip utilization: {schedule['max_strips_used']}/{config['total_strips']} strips")
    print(f"   Efficiency: {schedule['strips_efficiency']}")
    print(f"   Success rate: {schedule['scheduled_events']}/{schedule['total_events']} events")
    
    # Show strip assignments for a sample event
    if schedule['scheduled_events_details']:
        sample_event = list(schedule['scheduled_events_details'].values())[0]
        print(f"\n📍 Strip Assignment Example - {sample_event['event_name']}:")
        if sample_event.get('group'):
            print(f"   Group stage uses strips: #{', #'.join(map(str, sample_event['group']['strips']))}")
        if sample_event.get('knockout'):
            print(f"   Knockout uses strips: #{', #'.join(map(str, sample_event['knockout']['strips']))}")
    
    return schedule

Example 1: Running with sample data
📊 Using 10 sample events


AttributeError: 'OptimizedFencingScheduler' object has no attribute 'generate_schedule'